# [Argilla 101](https://docs.argilla.io/en/latest/index.html)
This notebook shows how to connect with the Argilla instance on the ET cluster. To run this notebook, you will need to provide the api key as the `ARGILLA_API_KEY` and the API URL as `ARGILLA_API_URL`  environment variables. You can place the key in a `credentials.env` at the root of this repository and this notebook will load it in the following cell. For an example, look at the `credentials_example.env` file.

In [1]:
import argilla as rg
from dotenv import load_dotenv, find_dotenv
import os
# Load environment variables
load_dotenv(find_dotenv("credentials.env"), override=True)

True

In [2]:
# Setup connection with the Argilla instance
rg.init(api_url=os.getenv("ARGILLA_API_URL"),
        api_key=os.getenv("ARGILLA_API_KEY"))

/opt/app-root/lib64/python3.8/site-packages/argilla/client/client.py:124: UserWarning: Default user was detected and no workspace configuration was provided, so the default 'argilla' workspace will be used. If you want to setup another workspace, use the `rg.set_workspace` function or provide a different one on `rg.init`
  warnings.warn(


In [3]:
# Create an example feedback dataset

dataset = rg.FeedbackDataset(
    guidelines="Add some guidelines for the annotation team here.",
    fields=[
        rg.TextField(name="prompt", title="Human prompt"),
        rg.TextField(name="output", title="Generated output", use_markdown=True)
    ],
    questions =[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the response:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]
        ),
        rg.TextQuestion(
            name="corrected-text",
            title="Provide a correction to the response:",
            required=False,
            use_markdown=True
        )
    ]
)

In [5]:
# Create an example record 
record = rg.FeedbackRecord(
    fields={
        "prompt": "Why can camels survive long without water?",
        "output": "Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time."
    }
)
# Add it to our feedback dataset
dataset.add_records(record)

In [7]:
# This publishes the dataset with its records to the Argilla instance for feedback annotation
remote_dataset = dataset.push_to_argilla(name="argilla-101", workspace="argilla")

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00, 41.51it/s]
/opt/app-root/lib64/python3.8/site-packages/argilla/client/feedback/dataset/mixins.py:195: DeprecationWarning: Calling `push_to_argilla` no longer implies that the `FeedbackDataset` can be updated in Argilla. If you want to push a `FeedbackDataset` and then update it in Argilla, you need to catch the returned object and use it instead: `remote_ds = ds.push_to_argilla(...)`. Otherwise, you can just call `push_to_argilla` and then `from_argilla` to retrieve the `FeedbackDataset` from Argilla, so the current `FeedbackDataset` can be retrieved as `FeedbackDataset.from_argilla(id='41f742ab-54dc-4e34-b25f-89de6acae78e')`.
  warnings.warn(


< Go to Argilla UI and annotate >

In [10]:
# Retrieve the annotated feedback dataset

feedback = rg.FeedbackDataset.from_argilla(
    name="argilla-101",
    workspace="argilla"
)

In [11]:
feedback

<FeedbackDataset id=41f742ab-54dc-4e34-b25f-89de6acae78e name=argilla-101 workspace=Workspace(id=e30eb6f7-6461-43a7-916d-e69c36807bea, name=argilla, inserted_at=2023-08-24 22:53:29.608650, updated_at=2023-08-24 22:53:29.608650) url=http://argilla-test-route-argilla-test.apps.et-cluster.6mwp.p1.openshiftapps.com/dataset/41f742ab-54dc-4e34-b25f-89de6acae78e/annotation-mode fields=[TextField(id=UUID('7d1989c5-3bb6-4d99-b211-cf01e27e5b03'), name='prompt', title='Human prompt', required=True, type='text', settings={'type': 'text', 'use_markdown': False}, use_markdown=False), TextField(id=UUID('ec5f967f-a6c9-4899-abab-41845eafcc1e'), name='output', title='Generated output', required=True, type='text', settings={'type': 'text', 'use_markdown': True}, use_markdown=True)] questions=[RatingQuestion(id=UUID('1b71bb30-2658-462e-acda-d138d7174d38'), name='rating', title='Rate the quality of the response:', description='1 = very bad - 5= very good', required=True, type='rating', settings={'type': 'r

In [12]:
# Extract example annotation
example_annotation = feedback.records[0].responses[0].values

rating = example_annotation['rating'].value
corrected_text = example_annotation['corrected-text'].value

print(f"The feedback rating (1-5) is {rating} and the corrected text is:'{corrected_text}'")

The feedback rating (1-5) is 2 and the corrected text is:'Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time. They can drink 50 gallons of water in one go. '


#### Next, we are going to use this notebook to create feedback collection for the ROSA docs.